In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install -qq -U diffusers==0.6.0 transformers ftfy gradio
!pip install git+https://github.com/huggingface/diffusers.git
!pip install omegaconf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.4/255.4 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 88.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 KB 28.5 MB/s eta 0:00:

In [8]:
import inspect
from typing import List, Optional, Union

import numpy as np
import torch

import PIL
from PIL import Image, ImageFilter, ImageOps, ImageDraw
import gradio as gr
from diffusers import StableDiffusionInpaintPipeline
from diffusers import EulerAncestralDiscreteScheduler, DDPMScheduler

import requests
from io import BytesIO

import os
import random
import tempfile
from omegaconf import OmegaConf
import subprocess

In [9]:
def image_grid(imgs, rows, cols):
    
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = PIL.Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    
    return grid

def image_grid(imgs, rows, cols):
    
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = PIL.Image.new('RGBA', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))

    line_img = PIL.Image.new('RGBA', size=(grid_w, grid_h), color=(255, 255, 255,0))
    draw = PIL.ImageDraw.Draw(line_img)

    for i in range(rows):
        y = (i+1)*h
        draw.line([(0, y), (grid_w, y)], width=2, fill=(0, 0, 0,255))

    for i in range(cols):
        x = (i+1)*w
        draw.line([(x, 0), (x, grid_h)], width=2, fill=(0, 0, 0,255))

    grid = PIL.Image.alpha_composite(grid, line_img)

    return grid



def get_data(data_path = 'gdrive/MyDrive/sd/stable-diffusion-data/manual run/test mask drawing woman'):
  
  images = []
  masks = []
  descriptions = []

  for filename in sorted(os.listdir(data_path)):
      if filename.endswith(".png"):
          if "-mask" in filename:
              masks.append(Image.open(os.path.join(data_path, filename)).resize((512, 512)))
          else:
              images.append(Image.open(os.path.join(data_path, filename)).resize((512, 512)))
      elif filename.endswith(".txt"):
          with open(os.path.join(data_path, filename), 'r') as file:
              descriptions.append(file.read())
  return images, masks, descriptions

def prepare_image_mask(image_mask, inpainting_mask_invert=True):

    image_mask = image_mask.convert('L')

    if inpainting_mask_invert:
        image_mask = ImageOps.invert(image_mask)

    return image_mask

def fill(image, mask, inpainting_fill='original'): #"fill" "original" "latent noise" "latent nothing"
    
    """fills masked regions with colors from image using blur. Not extremely effective."""
    if inpainting_fill == 'original':
      return image
    
    image_mod = Image.new('RGBA', (image.width, image.height))

    image_masked = Image.new('RGBa', (image.width, image.height))
    image_masked.paste(image.convert("RGBA").convert("RGBa"), mask=ImageOps.invert(mask.convert('L')))

    image_masked = image_masked.convert('RGBa')

    for radius, repeats in [(256, 1), (64, 1), (16, 2), (4, 4), (2, 2), (0, 1)]:
        blurred = image_masked.filter(ImageFilter.GaussianBlur(radius)).convert('RGBA')
        for _ in range(repeats):
            image_mod.alpha_composite(blurred)

    return image_mod.convert("RGB")

def dummy(images, **kwargs): return images, False 

def load_model(model_path = "runwayml/stable-diffusion-inpainting"):

  pipe = StableDiffusionInpaintPipeline.from_pretrained(
      model_path,
      revision="fp16", 
      torch_dtype=torch.float16,
      use_auth_token=True
  ).to(device)

  lms = EulerAncestralDiscreteScheduler.from_pretrained(model_path, subfolder="scheduler")

  pipe.scheduler = lms
  pipe.safety_checker = dummy

  return pipe

def get_row(pipe, prompt, image, mask_image, num_samples=3, CFG_Scale=7, seed=-1, num_inference_steps=20):

  if seed == -1:
    seed = random.randint(1, 9999999999)

  generator = torch.Generator(device="cuda").manual_seed(seed) # change the seed to get different results

  new_images = pipe(
      prompt=prompt,
      image=image,
      mask_image=mask_image,
      guidance_scale=CFG_Scale,
      generator=generator,
      num_images_per_prompt=num_samples,
      num_inference_steps=num_inference_steps
  ).images

  return new_images

In [10]:
num_row = 2
num_col = 3

path_init_folder = '/content'
path_models = '/content/gdrive/MyDrive/Dreambooth/Sessions/sideviewinp4VR'
data_path = '/content/gdrive/MyDrive/sd/stable-diffusion-data/manual run/test mask drawing woman'

device = "cuda"

path_temp_folder = path_init_folder + '/temp_data'
path_temp_model = path_init_folder + '/temp_model'
path_img_folder = path_models + '/out_imgs'

path_config_yaml = path_temp_folder + '/' + 'v1-inpainting.yaml'

In [11]:
!mkdir {path_temp_folder}
!mkdir {path_temp_model}
!mkdir {path_img_folder}

mkdir: cannot create directory ‘/content/temp_data’: File exists
mkdir: cannot create directory ‘/content/temp_model’: File exists
mkdir: cannot create directory ‘/content/gdrive/MyDrive/Dreambooth/Sessions/sideviewinp4VR/out_imgs’: File exists


In [4]:
%cd {path_temp_folder}
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
!wget https://raw.githubusercontent.com/V4-69/dreambooth_inpainting_c/main/v1-inpainting.yaml

/content/temp_data
--2023-01-12 23:48:50--  https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43051 (42K) [text/plain]
Saving to: ‘convert_original_stable_diffusion_to_diffusers.py’

convert_original_st 100%[===================>]  42.04K  --.-KB/s    in 0.01s   

2023-01-12 23:48:50 (3.27 MB/s) - ‘convert_original_stable_diffusion_to_diffusers.py’ saved [43051/43051]

--2023-01-12 23:48:51--  https://raw.githubusercontent.com/V4-69/dreambooth_inpainting_c/main/v1-inpainting.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.

In [12]:
images, masks, descriptions = get_data(data_path = data_path)
masks = [prepare_image_mask(image_mask, inpainting_mask_invert=True) for image_mask in masks]
images = [fill(image, mask, inpainting_fill='fill') for image, mask in zip(images, masks)]

In [13]:
model_name_list = [model_name for model_name in os.listdir(path_models) if model_name.endswith(".ckpt")]
model_name_list

['sideviewinp4-2200-16-v2-inpainting.ckpt',
 'sideviewinp4VR-201-16-4VR-inpainting.ckpt',
 'sideviewinp4VR-200-G5-16-VR-inpainting.ckpt']

In [14]:
model_name_list = [model_name for model_name in os.listdir(path_models) if model_name.endswith(".ckpt")]
for idx, model_name in enumerate(model_name_list):

  print(f'Model: {idx+1} of {len(model_name_list)}')
  print(model_name)
    
  %cd {path_temp_model}
  subprocess.call(f'rm -rf *', shell=True)
  %cd {path_temp_folder}

  print(f'Create temp model')
  path_model = path_models + '/' + model_name
  print(f'python convert_original_stable_diffusion_to_diffusers.py --checkpoint_path "{path_model}" --original_config_file "{path_config_yaml}" --dump_path "{path_temp_model}"')
  !python convert_original_stable_diffusion_to_diffusers.py --checkpoint_path "{path_model}" --original_config_file "{path_config_yaml}" --dump_path "{path_temp_model}" --scheduler_type "euler-ancestral"
  #subprocess.call(f'python convert_original_stable_diffusion_to_diffusers.py --checkpoint_path {path_model} --original_config_file {path_config_yaml} --dump_path {path_temp_model}')
  #subprocess.call(['python', 'convert_original_stable_diffusion_to_diffusers.py', '--checkpoint_path', path_model, '--original_config_file', path_config_yaml, '--dump_path', path_temp_model])
  print(f'Load temp model')
  pipe = load_model(model_path = path_temp_model)
  
  for idx_img in range(len(images)):

    print(f'Create imgs: {idx_img+1} of {len(images)}')
    
    description = descriptions[idx_img]
    image = images[idx_img]
    mask = masks[idx_img]
    new_images = [get_row(pipe, description, image, mask, num_samples=num_col, CFG_Scale=7, seed=-1) for _ in range(num_row)]
    new_images = np.concatenate(new_images)
    grid = image_grid(new_images, num_row, num_col)
    img_name = f'{idx_img:03d}_{model_name[:-5]}.png'
    img_path = path_img_folder + '/' + img_name
    grid.save(img_path)
  
  %cd {path_temp_model}
  subprocess.call(f'rm -rf *', shell=True)
  %cd {path_temp_folder}

  torch.cuda.empty_cache()

Model: 1 of 3
sideviewinp4-2200-16-v2-inpainting.ckpt
/content/temp_model
/content/temp_data
Create temp model
python convert_original_stable_diffusion_to_diffusers.py --checkpoint_path "/content/gdrive/MyDrive/Dreambooth/Sessions/sideviewinp4VR/sideviewinp4-2200-16-v2-inpainting.ckpt" --original_config_file "/content/temp_data/v1-inpainting.yaml" --dump_path "/content/temp_model"
global_step key not found in model
Downloading: 100% 4.52k/4.52k [00:00<00:00, 4.48MB/s]
Downloading: 100% 1.71G/1.71G [00:17<00:00, 99.7MB/s]
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.14.self_attn.q_proj.weight', 'vision_model.encoder.layers.2.mlp.fc1.weight', 'vision_model.encoder.layers.23.layer_norm2.weight', 'vision_model.encoder.layers.6.self_attn.out_proj.bias', 'vision_model.post_layernorm.bias', 'vision_model.encoder.layers.9.self_attn.v_proj.bias', 'vision_model.encoder.layers.7.mlp.fc1.weight', 

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Load temp model
Create imgs: 1 of 1


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Load temp model
Create imgs: 1 of 1


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

/content/temp_model
/content/temp_data
